In [1]:
cd ..

/mnt/WD_Blue/Machine_Learning/Proyectos/Chatbot-en-Espanol


In [2]:
from io import open
import time,os
import torch
import torch.nn as nn
import torch.quantization

from pre_processing import Voc
import load_transformer

torch.__version__ #torch > 1.3.0

'1.3.1'

In [3]:
model = load_transformer.from_checkpoint(load_quant=False)

quantized_model = torch.quantization.quantize_dynamic(
    model, {nn.LSTM, nn.Linear}, dtype=torch.qint8
)
print(quantized_model)

Loading: /mnt/WD_Blue/Machine_Learning/Proyectos/Chatbot-en-Espanol/data/Transformer_500k_UNK
Transformer(
  (encoder): Encoder(
    (embed): Embedder(
      (embed): Embedding(13762, 512)
    )
    (pe): PositionalEncoder(
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (layers): ModuleList(
      (0): EncoderLayer(
        (norm_1): Norm()
        (norm_2): Norm()
        (attn): MultiHeadAttention(
          (q_linear): DynamicQuantizedLinear(in_features=512, out_features=512, scale=1.0, zero_point=0)
          (v_linear): DynamicQuantizedLinear(in_features=512, out_features=512, scale=1.0, zero_point=0)
          (k_linear): DynamicQuantizedLinear(in_features=512, out_features=512, scale=1.0, zero_point=0)
          (dropout): Dropout(p=0.1, inplace=False)
          (out): DynamicQuantizedLinear(in_features=512, out_features=512, scale=1.0, zero_point=0)
        )
        (ff): FeedForward(
          (linear_1): DynamicQuantizedLinear(in_features=512, out_features=2048, sc

In [4]:
def print_size_of_model(model):
    torch.save(model.state_dict(), "temp.p")
    print('Size (MB):', os.path.getsize("temp.p")/1e6)
    os.remove('temp.p')

print_size_of_model(model)
print_size_of_model(quantized_model)

Size (MB): 262.044046
Size (MB): 108.817211


In [5]:
inputs = ['Hola','¿Cómo estás?','Pues hace buen día']

In [6]:
torch.set_num_threads(1)

def time_model_evaluation(model):
    s = time.time()
    for inp in inputs:
        load_transformer.evaluateOneInput(inp,model)
    elapsed = time.time() - s
    print(f"elapsed time (seconds): {elapsed:.2f}")

time_model_evaluation(model)
time_model_evaluation(quantized_model)

elapsed time (seconds): 1.90
elapsed time (seconds): 0.90


In [7]:
fn = os.path.join(load_transformer.save_dir, 'checkpoints',"transformer_quant_checkpoint.tar")
torch.save({
    'voc_dict': load_transformer.voc.__dict__,
    'params': quantized_model.state_dict()}, fn)